# Исследование надёжности заёмщиков

**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**В рамках проекта предстоит ответить на 4 вопроса:**
1. Есть ли зависимость между наличием детей и возратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на возврат кредита в срок?

**Проект будет состоять из 4 частей:**
1. Обзор данных
2. Предобработка данных
3. Ответ на поставленные вопросы
4. Вывод

## Шаг 1. Обзор данных

In [1]:
# импортируем библиотеку pandas
# прочитаем файл с данными для исследования
# посмотрим, как выглядит таблица

import pandas as pd

df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


При выводе первых 10 строк датасета сразу можно заметить проблемы в данных. 

В столбце `days_employed` встречаются отрицательные значения, отрицательных рабочих дней не существует. Вероятно, данные были неправильно переданы в систему или произошел какой-то сбой. 

Значения колонки `total_income` представляют собой средний доход. Для лучшего восприятия и работы с данными их следует перевести в целые числа.

В столбце `education` присутствуют неявные дубликаты. Например, среднее образование представлено разными значениями - "среднее", "Среднее", "СРЕДНЕЕ". Необходимо выявить все неявные дубликаты и избавиться от них.

In [2]:
# извлечем основную информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице 12 столбцов, 3 ключевых вида данных - `int64`, `float64`, `object`.

**Согласно документации к данным:**  
* `children` – количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

В названиях колонок нет нарушений стиля. 

Количество значений в данных разнится, что говорит нам о пропусках в данных в столбцах `days_employed` - общий трудовой стаж в днях, `total_income` - ежемесячный доход. Для дальнейшей работы с данными необходимо обработать пропуски. 

При первом обзоре данных уже можно выявить ошибки и неявные дубликаты в данных. В следующем шаге необходимо подготовить данные к исследованию, чтобы получить адекватные результаты исследования.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
# подсчитаем количество пропусков в каждом столбце
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Для подсчета пропусков применили метод `isna()`. А чтобы подсчитать пропуски воспользуемся методом `sum()`.

Пропуски встречаются в двух колонках `days_employed` и `total_income`. Вероятно, пропуски в столбцах взаимосвязаны - если нет данных о стаже пользователя в днях, не будет и данных о его доходах. Данное предположение исходит из количества пропусков в обоих столбцах, оно одинаковое.

Данные в столбце `total_income` являются важными для ответа на один из поставленных вопросов в ходе исследования - "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?", поэтому мы не можем заменить пропуски на 0. Избавиться от строк с пропущенными значениями этого столбца мы тоже не можем, так как пропуски составляют около 10% от всех данных и могут существенно повлиять на ход исследования и на ответы на другие вопросы.

Заменим пропуски в столбце `total_income` на медианные значения ежемесячных доходов, исходя из типа занятости респондента. Выбираем именно медианное значение, а не среднее арифметическое, так как в данных об уровне дохода могут быть выбросы в большую и меньшую сторону, что может привести к неверным расчетам.

Данные в столбце `days_employed` не являются важными для ответа ни на один из поставленных вопросов, поэтому этот столбец удаляем из таблицы.

Посмотрим, сколько уникальных значений содержит столбец `income_type`. Для этого вызовем метод **`.unique()`**. Заодно проверим, чтобы в этом столбце не было неявных дубликатов.

In [4]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В столбце `income_type` респонденты разделены на 8 типов занятости: сотрудник, пенсионер, компаньон, госслужащий, безработный, предприниматель, студент, в декрете. Заменим пропущенные значения на медианные в разрезе типа занятости с помощью метода **`.transform()`**

In [5]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

Проверим, остались ли пропущенные значения в данных в столбце `total_income`.

In [6]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Так как данные в столбце `days_employed` не несут сущетсвенного значения для ответа на поставленные вопросы в ходе исследования. Можно удалить столбец, однако всегда есть вероятность, что данные пригодятся в дальнейшем. Столбец оставим без изменений в датасете, при необходимости использования данных этого столбца, он будет преобразован и приведен к удовлетворительному для исследования виду.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Первичная обработка пропусков завершена. С помощью методов `isna()` и `sum()` были обнаружены и посчитаны пропуски в датасете. Пропуски могут быть связаны с ошибкой хранения данных, некоторые значения могли быть утеряны. Также вероятно, что клиенты банка могли не указать такие данные о себе. 

Следующим шагом пропуски были заменены на медианные значения в зависимости от типа занятости клиентов. Это позволило избежать удаления практичести 10% важных для анализа сведений. Замена на нулевые значения не представлялась возможной, так как могла сильно исказить результаты исследования в ответе на один из поставленных вопросов. 

Данные столбца `total_income` все еще не являются до конца пригодными для качественного исследования. Следующим этапом предстоит перевести данные из вещественного типа в целочисленный.

Столбец `days_employed` содержал отрицательные значения, что противоречит здравому смыслу, стаж работы клиентов не может быть представлен в виде отрицательных значений. Причиной такого отображения данных может быть ошибка в обработке базы данных. Было принято решение оставить этот столбец без изменений. В случае необходимости использования данных для исследования, столбец будет обработан должным образом.

### Замена типа данных

Необходимо изменить тип данных в столбце `total_income` с `float` на `int`. Применим метод `astype()`, в качестве аргумента подставим `int`.

In [8]:
df.loc[:, 'total_income'] = df.loc[:, 'total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [9]:
# посмотрим, как теперь выглядят данные в столбце total_income

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

В этом шаге мы заменили тип данных столбца `total_income` методом `astype()`, в качестве аргумента подставили `int` - целые числа. Данный метод позволяет заменить значения в нужный тип. Данная операция была проведена для лаконичности представления значений и дальнейших подсчетов.

### Обработка дубликатов

Для начала выявим явные дубликаты. Найдем их с помощью метода `duplicated()` и найдем общее количество дубликатов.

In [10]:
df.duplicated().sum()

54

С помощью логической индексацией получим таблицу с дубликатами и посмотрим на первые 10 строк.

In [11]:
df_duplicated = df[df.duplicated()].head(10)
df_duplicated

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,NaN,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,NaN,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,NaN,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,NaN,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,NaN,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


Избавимся от явных дубликатов с помощью метода `drop_duplicates()`, обновим индексацию через `reset_index()` и удалим старые индексы, применив параметр `drop=True`.

In [12]:
df = df.drop_duplicates().reset_index(drop=True)

Ещё раз проверим датафрейм на наличие явных дубликатов.

In [13]:
df.duplicated().sum()

0

Мы избавились от всех явных дубликатов. Время проверить датафрейм на наличие неявных дубликатов.

При первом выводе строчек датафрейма были найдены неявные дубликаты в столбце `education`. Посмотрим, сколько уникальных значений содержит этот столбец. Вызовем метод `unique`. 

In [14]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В столбце представлено 5 типов образования клиента, но уникальных названий 15. Так тип образования представлен в 3 уникальных значениях каждый. Приведем их к единому стилю и исправим альтернативные написания значений. 

Можно заменить значения с помощью метода `replace()`, а для автоматизации процесса написать функцию, которой будем передавать в качестве первого аргумента неверное значение (список неверных значений), а в качестве второго - корректное значение.

Но есть вариант, который заметно сократит время работы кода и уместится в одну строчку - строковый метод `str.lower()`, который можем применить ко всему столбцу датафрейма.

In [15]:
df['education'] = df['education'].str.lower()

In [16]:
# снова вызовем метод unique() и проверим уникальные значения

df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Мы избавились от всех неявных дубликатов в столбце `education`. Проверим значения других столбцов, которые могут повлиять на качество проводимого нами исследования.

Рассмотрим столбцы `children`, `dob_years` и `family_status`. Применим метод `unique()`.

In [17]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Значения -1 и 20 явно являются ошибочными: не может быть отрицательное количество детей, а 20 - слишком большое число.

Посмотрим, какое распределение количества детей среди клиентов банка при помощи метода `value_counts()`. Если количество ошибочных значений небольшое и составит меньше 1% от всех данных, то строки с такими значениями можно удалить или заменить на наиболее реалистичные. 

In [18]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Всего ошибочных значений 123, что составляет примерно 0,5% от всех данных. Удаление или замена таких значений существенно не повлияет на результаты исследования. 

Вероятно, такие данные появились по ошибке программы. Предположим, что -1 - это 1 ребенок в семье, а 20 - 2 ребенка. Заменим некорректные значения в столбце с помощью метода `replace()`.

In [19]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

In [20]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Теперь данные о количестве детей выглядят реалистичнее. 

Посмотрим, есть ли неявные дубликаты в столбце `family_status`.

In [21]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Неявные дубликаты обнаружены не были, но не все значения записаны в едином стиле. Исправим это с помощью метода `replace()`.

In [22]:
df['family_status'] = df['family_status'].replace('Не женат / не замужем', 'не женат / не замужем')

In [23]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Теперь все значения столбца приведены к единому стилю.

Проверим, если ли неявные дубликаты или ошибочные значения в столбце `dob_years`.

In [24]:
df['dob_years'].value_counts()

35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
56    484
52    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
55    443
59    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце обнаржены клиенты с нулевым возрастом, что является явной ошибкой. Количество таких значений составляет меньше 0,5%. Этот столбец никак не участвует в решении поставленных задач, поэтому данные можно оставить без изменения, на ход исследования они не повлияют.

Проверим преобразованный датафрейм на наличие явных дубликатов. Возможно, в ходе работы с ошибочными значениями и неявными дубликатами какие-то строки стали дублированными.

In [25]:
df.duplicated().sum()

17

В таблице снова появились дубликаты. Это может быть связано с преобразованием значений в столбце `education`. Например, было две строчки с одинаковыми значениями во всех столбцах, но в типе образования не совпадали значения, потому что в одной строке было значение 'ВЫСШЕЕ',  а в другой 'Высшее'. Избавимся от дубликатов.

In [26]:
df = df.drop_duplicates().reset_index(drop=True)

In [27]:
df.duplicated().sum()

0

**Вывод**

Была проведена работа с явными и неявными дубликатами, а также ошибочными значениями в данных.

С помощью метода `duplicated()` и метода `sum()` были обнаружены дубликаты датасета. Явные дубликаты были удалены с помощью метода `drop_duplicates()`, также была обновлена индексацию через `reset_index()` и удалены старые индексы. Для удаления старых индексов применили параметр `drop=True`.

На неявные дубликаты проверяли столбцы `education`, `children` и `family_status`, применив метод `unique()`. Данный метод позволяет посмотреть на уникальные значения столбца. 

В столбце `education` заменили неявные дубликаты, написав отдельную функцию `replace_wrong_values(wrong_values, correct_value)`, которая заменяет неправильное значение на корректное.

При вызове метода `unique()` для столбца `children` были обнаружены несвойственные этому столбцу значения - -1 и 20. Вероятно, данные были внесены по ошибке. Предположили, что -1 - это 1 ребенок, а 20 - это 2 ребенка, и преобразовали столбец, вызвав метод `replace()`, где в качестве первого аргумента поставили некорректное значение, а в качестве второго - то, на которое приняли решение это значение заменить.

Столбец `family_status` не содержал неявные или ошибочные значения, но данные были представлены не в едином стиле. Преобразовали столбец также с помощью метода `replace()`.

`dob_years` содержал нулевые значения, что не соответствует действительности. Процент таких значений меньше 0,5, поэтому было принято решение оставить эти данные без изменения, тем более, что данные не участвуют в исследовании.

В конце еще раз проверили датасет на явные дубликаты и избавились еще от 17 строк, которые появились в ходе работы с неявными дубликатами и ошибочными значениями.

### Лемматизация

Для ответа на один из поставленных в ходе исследования вопросов, нам необходимо выяснить причину, по которой клиенты брали кредит. Посмотрим, какие значения содержит столбец `purpose`.

In [28]:
display(df['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Все причины можно сгруппировать по общей цели кредита, используя лемматизацию. 

Однако, если бы причин было больше, было бы практически невозможно их категоризировать и визуально оценить. Оценим частотность встречаемости отдельных лемм в столбце `purpose`. Для этого ипортируем библиотеку `pymystem3` и  специальный контейнер `Counter` из модуля `collections`.

In [29]:
# импортируем библиотеку pymystem3

from pymystem3 import Mystem
m = Mystem()

In [30]:
# импортируем контейнер Counter из модуля collections

from collections import Counter

In [31]:
lemmas_list = []
for lemma in df['purpose']:
    lemmas = ''.join(m.lemmatize(lemma)).strip()
    lemmas_list.append(lemmas)
print(Counter(lemmas_list))


Counter({'автомобиль': 972, 'свадьба': 791, 'на проведение свадьба': 768, 'сыграть свадьба': 765, 'операция с недвижимость': 675, 'покупка коммерческий недвижимость': 661, 'операция с жилье': 652, 'покупка жилье для сдача': 651, 'операция с коммерческий недвижимость': 650, 'покупка жилье': 646, 'жилье': 646, 'покупка жилье для семья': 638, 'строительство собственный недвижимость': 635, 'недвижимость': 633, 'операция со свой недвижимость': 627, 'строительство жилой недвижимость': 624, 'покупка недвижимость': 621, 'покупка свой жилье': 620, 'строительство недвижимость': 619, 'ремонт жилье': 607, 'покупка жилой недвижимость': 606, 'на покупка свой автомобиль': 505, 'заниматься высокий образование': 496, 'сделка с подержанный автомобиль': 486, 'на покупка подержать автомобиль': 478, 'свой автомобиль': 478, 'на покупка автомобиль': 471, 'приобретение автомобиль': 461, 'дополнительный образование': 460, 'сделка с автомобиль': 455, 'высокий образование': 452, 'образование': 447, 'получение до

In [32]:
# напишем цикл, который будет с помощью лемматизации категоризовать данные

def lemmatize_purpose(row):
    purpose_category = m.lemmatize(row['purpose'])
    if 'свадьба' in purpose_category:
        return 'свадьба'
    if ('жилье' in purpose_category) or ('недвижимость' in purpose_category):
        return 'недвижимость'
    if 'автомобиль' in purpose_category:
        return 'автомобиль'
    if 'образование' in purpose_category:
        return 'образование'

**Вывод**

В этом шаге мы создали функцию, в которой применили лемматизацию к значениям в столбце `purpose` и сгруппировали данные по общей цели кредита. Это нам понадобится в дальнейшем для категоризации по столбцу цель кредита и ответа на один из вопросов исследования. 

### Категоризация данных

In [33]:
# добавим к датафрейму новый столбец с отсортированными по цели взятия кредита данными

df['purpose_category'] = df.apply(lemmatize_purpose, axis=1)

In [34]:
# посмотрим, распределение клиентов по целям кредита

df['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

Разделим столбец `children` на категории. Для этого напишем функцию, которая будет определять значение 0 как семьи без детей, 1-2 - семьи с детьми и 3 и более - многодетные семьи.

In [35]:
def children_category(child):
    if child == 0:
        return 'нет детей'
    if 1 <= child < 3:
        return 'есть дети'
    return 'многодетные'

# добавим к датафрейму столбец с категориями семей в разрезе количества детей

df['children_id'] = df['children'].apply(children_category)

# посмотрим на первые 10 строк датафрейма с новым столбцом

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,есть дети
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,нет детей
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,нет детей
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,есть дети
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей


Разделим столбец `total_income` на 5 категорий заработной платы. Для начала посмотрим на минимум, максимум и медианное значение уровня ежемесячного дохода по всему датафрейму.  

In [36]:
min_income = df['total_income'].min()
min_income

20667

In [37]:
max_income = df['total_income'].max()
max_income

2265604

In [38]:
median_income = df['total_income'].median()
median_income

142594.0

In [39]:
mean_income = df['total_income'].mean()
mean_income.round()

165320.0

Минимальное значение зработной платы - 20 667. Средняя зарплата по России на 2020-2021 год составляет около 54 000 рублей. Условимся, что доход ниже этого уровня - низкий уровень дохода (в эту категорию попадут значения от минимума до 54 000 рублей). Средний уровень дохода будем считать от 54 000 рублей до 85 000 рублей. В категорию выше среднего попадут значения от 85 000 рублей до 140 000 рублей. Высокой будет считаться зарплата, которая присуща руководящему звену (от 140 до 350 тысяч рублей). Все, что выше распределим в категорию сверхвысокий уровень дохода. 

Напишем функцию, которая разделит доходы клиентов банка на категории.

In [40]:
def income_category(income):
    if income < 54000:
        return 'ниже среднего'
    if 54000 <= income < 85000 :
        return 'средний'
    if 85000 <= income < 140000:
        return 'выше среднего'
    if 140000 <= income < 350000:
        return 'высокий'
    return 'сверхвысокий'

In [41]:
# создадим новый столбец, в который попадут значения total_income с применением функции

df['income_id'] = df['total_income'].apply(income_category)

In [42]:
# выведем первые 10 строк датафрейма

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_id,income_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,есть дети,высокий
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети,выше среднего
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,высокий
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные,высокий
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,высокий


**Вывод**

Мы разделили на категории 3 столбца `purpose`, `children` и `total_income`. Это необходимо для дальнейшего упрощения анализа данных при ответе на поставленные вопросы.

В столбце `purpose` сначала применили лемматизацию, а затем добавили новый столбец в датасет с категориями целей кредита (всего их получилось 4: 1) недвижимость, 2) автомобиль, 3) образование и 4) свадьба.

Для столбца `children` написали функцию, которая принимает в качестве аргумента значение столбца, а на выходе определяет название категории. Получилось 3 категории: 1) нет детей, 2) есть дети, 3) многодетные.

`total_income` отражает уровень дохода. Этот столбец был разделен на 5 групп: 1) ниже среднего, 2) средний, 3) выше среднего, 4) высокий, 5) сверхвысокий.

## Шаг 3. Ответы на вопросы исследвания

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [43]:
# создадим сводную таблицу

pivot_table_children = df.pivot_table(index='children_id', columns='debt', 
                                      values='total_income', aggfunc='count')

In [44]:
# добавим столбец, который считает процентное соотношение клиентов, непогасивших задолженность 
# к общему числу клиентов

pivot_table_children['percent'] = (pivot_table_children[1] / 
                                   (pivot_table_children[1] + pivot_table_children[0]) * 100)

# округлим значения до одного символа после запятой

pivot_table_children['percent'] = pivot_table_children['percent'].round(1)

In [45]:
# выведем таблицу, отсортировав значения по убыванию по столбцу percent

pivot_table_children.sort_values('percent', ascending=False)

debt,0,1,percent
children_id,,,
есть дети,6336,647,9.3
многодетные,349,31,8.2
нет детей,13028,1063,7.5


**Вывод**

Наличие детей влияет на возврат кредита в срок. 9,3% клиентов с детьми не погасиили задолженность в срок, тогда как из общего числа клиентов без детей не погасили задолженность в срок только 7,5%. Возможно, в семьях с детьми существует большая вероятность незапланированных расходов, что приводит к усложненному процессу планирования бюджета и, в следствие, просроченных выплат по погашению кредита.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [46]:
pivot_table_family_status = df.pivot_table(index='family_status', columns='debt', 
                                           values='total_income', aggfunc='count')
pivot_table_family_status['percent'] = (pivot_table_family_status[1] /
                                        (pivot_table_family_status[1] + pivot_table_family_status[0]) *100)
pivot_table_family_status['percent'] = pivot_table_family_status['percent'].round(1)
pivot_table_family_status.sort_values('percent', ascending=False)

debt,0,1,percent
family_status,,,
не женат / не замужем,2536,274,9.8
гражданский брак,3763,388,9.3
женат / замужем,11408,931,7.5
в разводе,1110,85,7.1
вдовец / вдова,896,63,6.6


**Вывод**

Клиенты, не состоящие в браке, более склонны к просроченным задолженностям (9,8%), на ряду с ними - клиенты, состоящие в гражданском браке (9,3%). Это может быть вызвано тем, что такие клиенты зависят только от своих доходов и не могут положиться на помощь супруга/супруги. Такие задолженности людям сложнее погашать в одиночку. 

У клиентов, состоящих в браке, показатель возврата кредита выше на ~2%. Это можно обосновать наличием общего бюджета, а также финансового вовлечение с двух сторон, что приводит к вовремя погашенному долгу.

В категории "в разводе" и "вдовец/вдова" маленькая выборка клиентов для формулирования обоснованных выводов.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [47]:
pivot_table_income = df.pivot_table(index='income_id', columns='debt', 
                                    values='total_income', aggfunc='count')
pivot_table_income['percent'] = (pivot_table_income[1] / 
                                 (pivot_table_income[1] + pivot_table_income[0]) *100)
pivot_table_income['percent'] = pivot_table_income['percent'].round(1)
pivot_table_income.sort_values('percent', ascending=False)

debt,0,1,percent
income_id,,,
выше среднего,6128,570,8.5
высокий,10199,905,8.2
средний,2090,177,7.8
ниже среднего,492,34,6.5
сверхвысокий,804,55,6.4


**Вывод**

Процент клиентов, погасивших в срок задолженность, не зависит напрямую от уровня их доходов. Логично было бы предположить, что те, у кого доходы выше, своевременнее возвращают кредит в срок. Данные говорят об обратном. На удивление клиенты с высоким доходом и доходом выше среднего чаще остальных групп имеют проблемы со своевременным погашением кредита в срок. Клиенты со средним доходом имеют более хорошую возвратную историю. Можно это связать с тем, что такие клиенты хорошо умеют жить с урезанным бюджетом, умеют правильно планировать расходы и чаще откладывают деньги на особый случай.

Клиентов со сверхвысоким доходом и доходом ниже среднего по отношению к другим группам достаточно маленькое количество. На основании этих данных нельзя сделать точные выводы.

- **Как разные цели кредита влияют на его возврат в срок?**

In [48]:
pivot_table_purpose = df.pivot_table(index='purpose_category', columns='debt', 
                                     values='total_income', aggfunc='count')
pivot_table_purpose['percent'] = (pivot_table_purpose[1] / 
                                  (pivot_table_purpose[1] + pivot_table_purpose[0]) *100)
pivot_table_purpose['percent'] = pivot_table_purpose['percent'].round(1)
pivot_table_purpose.sort_values('percent', ascending=False)

debt,0,1,percent
purpose_category,,,
автомобиль,3903,403,9.4
образование,3643,370,9.2
свадьба,2138,186,8.0
недвижимость,10029,782,7.2


**Вывод**

Кредит на недвижимость и связанные с ней действия является самой распространенной целью среди клиентов банка. Клиенты, относящиеся к данной категории, вероятно осознаннее относятся к планированию своего бюджета, в следствие чего чаще закрывают задолженность в срок. Это может быть связано с тем, что такая цель достаточно масштабна и требует больших вложений, что приводит к более тщательному и взвешенному отношению со стороны клиентов.

Образование часто является вынужденной мерой взятия кредита, например, на образование детей в высшем учебном заведении. Доход и срочность получения услуги не всегда соизмеримы. Это может быть причиной бóльшего процента клиентов, не погасивших задолженность в срок (9,2%).

Клиенты, получившие кредит с целю покупки автомобиля, чаще остальных не выплачивают своевременно долг (9,4%).

## Шаг 4. Общий вывод

В проекте было поставлено 4 вопроса, при ответе на которые выяснилось:

1. Зависимость между наличием детей и возвратом кредита в срок существует. Клиенты с детьми чаще просрачивают выплаты по кредиту в срок, чем клиенты без детей.
* **9,3%** клиентов с 1-2 детьми, не вернувших кредит вовремя;
* **8,2%** многодетных клиентов, не вернувших кредит вовремя;
* **7,5%** клиентов без детей, не вернувших кредит вовремя.

2. Зависимость между семейным положением и возвратом кредита в срок есть. Клиенты, не состоящие в браке или состоящие в гражданском браке, чаще не погашают долг в срок по сравнению с клиентами, состоящими в браке.
* **9,8%** клиентов, не состоящих в браке, не вернули кредит вовремя;
* **9,3%** клиентов, состоящих в гражданском браке, не вернули кредит вовремя;
* **7,5%** клиентов, состоящих в браке, не вернули кредит вовремя.

3. Зависимость между уровнем дохода и возвратом кредита в срок есть, но она не такая значительная.
* **8,5%** клиентов с уровнем дохода выше срденего, не погасивших кредит в срок;
* **8,2%** клиентов с высоким уровнем дохода, не погасивших кредит в срок;
* **7,8%** клиентов со средним уровнем дохода, не погасивших кредит в срок.

4. Цели кредита влияют на возврат кредита в срок. Клиенты, оформившие кредит на покупку или другие операции с недвижимостью, лучше остальных категорий выплачивают долг **(7,2% должников)**. Такие цели как покупка автомобиля или оплата образования чаще приводят к задолженностям по выплатам со стороны клиентов **(9,4% и 9,2% должников соответственно)**. Кредит на свадьбу берут меньшее количество клиентов, а людей, не погасивших в срок задолженность, - **8%**.

**Можем сделать вывод, что чаще других погашают задолженность в срок клиенты без детей, состоящие в браке, со средним уровнем дохода, с целью покупки недвижимости или иных операций с ней.**